In [40]:
import gmaps
import pandas as pd

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [41]:
crime_df = pd.read_csv('Resources/KCPD_Crime_Data.csv')
crime_df.dropna(inplace = True) 
crime_df.dropna()
crime_df

,Report_No,Reported_Date,Reported_Time,From_Date,From_Time,To_Date,To_Time,Offense,IBRS,Description,...,Zip Code,Rep_Dist,Area,DVFlag,Involvement,Race,Sex,Age,Firearm Used Flag,Location
384,KC20011765,02/15/2020,2020-02-15T20:03:38.000,02/10/2020,2020-02-10T12:00:00.000,02/15/2020,2020-02-15T15:00:00.000,Stolen Auto,23F,Theft From Motor Vehicle,...,64106.0,PJ0696,CPD,N,VIC,W,M,60.0,False,W 8TH ST and MULBERRY ST\nKANSAS CITY 64106\n(...
385,KC20012815,02/20/2020,2020-02-20T06:55:03.000,02/19/2020,2020-02-19T00:01:00.000,02/20/2020,2020-02-20T06:50:00.000,Stealing from Auto (Theft from Auto),23F,Theft From Motor Vehicle,...,64111.0,PJ3372,CPD,N,VIC,W,F,23.0,False,"4100 OAK ST\nKANSAS CITY 64111\n(39.051476, -..."
398,KC20007266,01/29/2020,2020-01-29T16:25:56.000,01/29/2020,2020-01-29T05:30:00.000,01/29/2020,2020-01-29T16:50:00.000,Burglary (Residential),220,Burglary/Breaking and Entering,...,64129.0,PJ4134,MPD,N,SUS,B,M,22.0,False,7600 SNI-A-BAR RD\nKANSAS CITY 64129\n(39.035...
410,KC20008528,02/03/2020,2020-02-03T12:10:14.000,12/17/2019,2019-12-17T03:52:00.000,02/03/2020,2020-02-03T12:10:00.000,Identity Theft,26F,Identity Theft,...,64151.0,PP0691,NPD,N,VIC,W,M,59.0,False,5700 N HUTSON RD\nKANSAS CITY 64151\n(39.1987...
419,KC20008819,02/04/2020,2020-02-04T11:21:03.000,02/02/2020,2020-02-02T11:34:00.000,02/04/2020,2020-02-04T10:29:00.000,Violation of Ex-Parte Order of Protection,90Z,All Other Offenses,...,64114.0,PJ6193,MPD,Y,VIC,B,F,30.0,False,8700 WORNALL RD\nKANSAS CITY 64114\n(38.97066...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47953,KC20043689,07/02/2020,2020-07-02T07:45:29.000,07/01/2020,2020-07-01T17:00:00.000,07/02/2020,2020-07-02T07:00:00.000,Stealing from Auto (Theft from Auto),23F,Theft From Motor Vehicle,...,64127.0,PJ0983,EPD,N,VIC,W,M,53.0,False,2400 E 11TH ST\nKANSAS CITY 64127\n(39.100736...
47978,KC20044172,07/04/2020,2020-07-04T10:47:37.000,07/03/2020,2020-07-03T11:00:00.000,07/04/2020,2020-07-04T10:47:00.000,Burglary (Residential),220,Burglary/Breaking and Entering,...,64134.0,PJ6670,SPD,N,VIC,I,F,43.0,False,8500 E 99TH ST\nKANSAS CITY 64134\n(38.944361...
47982,KC20043923,07/03/2020,2020-07-03T07:39:59.000,07/02/2020,2020-07-02T17:00:00.000,07/03/2020,2020-07-03T07:20:00.000,Property Damage,290,Vandalism/Destruction of Property,...,64153.0,PP0046,NPD,N,CMP VIC,W,M,30.0,False,12200 N POMONA LN\nKANSAS CITY 64153\n(39.315...
47984,KC20043364,06/30/2020,2020-06-30T20:36:54.000,06/30/2020,2020-06-30T20:30:00.000,06/30/2020,2020-06-30T20:35:00.000,Domestic Violence Assault (Aggravated),13A,Aggravated Assault,...,64130.0,PJ4289,MPD,Y,VIC,W,F,22.0,False,5300 BROOKLYN AVE\nKANSAS CITY 64130\n(39.030...


In [42]:
location = crime_df["Location"]= crime_df["Location"].str.split("(", n = 1, expand = True) 

location

,0,1
384,W 8TH ST and MULBERRY ST\nKANSAS CITY 64106\n,"39.105094, -94.599678)"
385,4100 OAK ST\nKANSAS CITY 64111\n,"39.051476, -94.581738)"
398,7600 SNI-A-BAR RD\nKANSAS CITY 64129\n,"39.035286, -94.495582)"
410,5700 N HUTSON RD\nKANSAS CITY 64151\n,"39.198787, -94.620837)"
419,8700 WORNALL RD\nKANSAS CITY 64114\n,"38.970664, -94.59528)"
...,...,...
47953,2400 E 11TH ST\nKANSAS CITY 64127\n,"39.100736, -94.553634)"
47978,8500 E 99TH ST\nKANSAS CITY 64134\n,"38.944361, -94.491134)"
47982,12200 N POMONA LN\nKANSAS CITY 64153\n,"39.315115, -94.671276)"
47984,5300 BROOKLYN AVE\nKANSAS CITY 64130\n,"39.030194, -94.559609)"


In [43]:
location = location[1].str.split(",",expand=True)

In [44]:
location[1] = location[1].str.replace(')', '')


In [45]:
renamed= location.rename(columns={0: "Lat", 1:"Lng"})
renamed

,Lat,Lng
384,39.105094,-94.599678
385,39.051476,-94.581738
398,39.035286,-94.495582
410,39.198787,-94.620837
419,38.970664,-94.59528
...,...,...
47953,39.100736,-94.553634
47978,38.944361,-94.491134
47982,39.315115,-94.671276
47984,39.030194,-94.559609


In [46]:
coordinates= renamed[["Lat","Lng"]]
coordinates['weight']=1
coordinates

,Lat,Lng,weight
384,39.105094,-94.599678,1
385,39.051476,-94.581738,1
398,39.035286,-94.495582,1
410,39.198787,-94.620837,1
419,38.970664,-94.59528,1
...,...,...,...
47953,39.100736,-94.553634,1
47978,38.944361,-94.491134,1
47982,39.315115,-94.671276,1
47984,39.030194,-94.559609,1


In [47]:
coordinates["Lat"]= pd.to_numeric(coordinates["Lat"])

In [48]:
coordinates["Lng"]= pd.to_numeric(coordinates["Lng"])

In [51]:
records= coordinates[['Lat','Lng']].values.tolist()
records

[[39.105094, -94.599678],
 [39.051476, -94.581738],
 [39.035286, -94.495582],
 [39.198787, -94.620837],
 [38.970664, -94.59528],
 [39.225092, -94.648454],
 [38.926497999999995, -94.51896500000001],
 [39.069476, -94.541173],
 [39.033747, -94.41578],
 [39.033747, -94.41578],
 [38.942021000000004, -94.607128],
 [38.983381, -94.6016],
 [39.152142, -94.47720799999999],
 [39.090303000000006, -94.512662],
 [39.041419, -94.572964],
 [39.056940999999995, -94.547814],
 [39.10003, -94.585827],
 [39.248754, -94.593197],
 [39.116993, -94.526883],
 [39.069121, -94.585387],
 [39.152142, -94.47720799999999],
 [39.026677, -94.533668],
 [38.958465000000004, -94.488186],
 [39.093106, -94.533265],
 [39.074714, -94.547993],
 [39.051373, -94.595768],
 [39.104748, -94.50686999999999],
 [39.106774, -94.56390400000001],
 [38.971462, -94.605386],
 [39.110335, -94.580974],
 [39.070268, -94.571333],
 [39.258571999999994, -94.526178],
 [39.069128000000006, -94.529706],
 [38.975286, -94.492262],
 [39.102357, -94.58

In [52]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(records, weights = coordinates['weight'],
                                 dissipating=False, max_intensity=50,
                                 point_radius=0.01)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))